In [1]:
import pandas as pd, numpy as np, warnings
import datetime
from datetime import date
pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

import xgboost as xgb
from xgboostlss.distributions import *
from xgboostlss.distributions.distribution_utils import DistributionClass

from xgboostlss.model import *
from xgboostlss.distributions.Gamma import *
from xgboostlss.distributions.Weibull import *
from xgboostlss.distributions.Expectile import *

from sklearn import datasets
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import mean_absolute_error as mae, mean_squared_error as mse, mean_absolute_percentage_error as mape

In [2]:
segs = pd.read_parquet('final_segs.parquet')
segs['MEDIAN_TYPE'] = segs['MEDIAN_TYPE'].fillna(8)
segs['MEDIAN_WIDTH'] = segs['MEDIAN_WIDTH'].fillna(0)
segs.head()

,LNSegID,ROUTE_ID,BEGIN_MP,END_MP,FS_SYSTEM,RURAL_URBAN,AT_GRADE_OTHER,AT_GRADE_SIGNALS,AT_GRADE_SIGNS,DegHCurve_LWA,DegGrade_LWA,AADT,DirAADT,PEAK_CAPACITY_HCM6,TRUCK_PERC,TYPE_TERRAIN,THROUGH_LANES,LANE_WIDTH,SHLD_WIDTH_L,SHLD_WIDTH_R,SPEED_LIMIT_LWA,MEDIAN_TYPE,MEDIAN_WIDTH,SECTION_LENGTH,INTERCHANGE_SEG
0,1,001-LN-9008 -000,36.401,46.302,2,Rural,0,0,0,1.7,2.608974,4386,2193.0,1805.0,27.247,2.0,4.0,12.0,6.0,10.0,70.0,2.0,36.0,9.901,No
1,2,001-LN-9008 -000,46.942,48.537,2,Rural,0,0,0,1.7,1.450000,8643,4322.0,1652.0,25.012,2.0,4.0,12.0,6.0,10.0,70.0,2.0,36.0,1.595,No
2,3,001-LN-9008 -000,49.183,57.552,2,Rural,0,0,0,1.7,1.450000,7953,3976.0,1593.0,25.012,2.0,4.0,12.0,6.0,10.0,70.0,2.0,36.0,8.369,No
3,4,003-BG-9002 -000,45.039,47.596,2,Rural,0,0,0,1.7,1.632245,16235,8118.0,3501.0,13.172,1.0,4.0,12.0,5.0,10.0,70.0,2.0,36.0,2.557,No
4,5,003-BG-9002 -000,48.031,58.609,2,Rural,0,0,0,1.7,2.095735,12370,6185.0,2782.0,28.638,1.0,4.0,12.0,5.0,10.0,70.0,2.0,36.0,10.578,No


In [24]:
segs.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
LNSegID,1445.0,723.000000,417.279882,1.000,362.000,723.000000,1084.00000,1445.000
BEGIN_MP,1445.0,59.667866,52.010686,0.000,16.123,42.853000,94.15400,191.544
END_MP,1445.0,60.630186,52.040170,0.054,16.792,43.424000,94.43800,191.777
FS_SYSTEM,1445.0,1.254671,0.454495,1.000,1.000,1.000000,1.00000,4.000
AT_GRADE_OTHER,1445.0,0.001384,0.037190,0.000,0.000,0.000000,0.00000,1.000
AT_GRADE_SIGNALS,1445.0,0.000000,0.000000,0.000,0.000,0.000000,0.00000,0.000
AT_GRADE_SIGNS,1445.0,0.000692,0.026307,0.000,0.000,0.000000,0.00000,1.000
DegHCurve_LWA,1445.0,1.747247,0.324807,1.700,1.700,1.700000,1.70000,6.950
DegGrade_LWA,1445.0,2.304833,1.053326,0.200,1.450,1.994379,3.27699,7.450
AADT,1445.0,49390.594464,38487.031497,3000.000,18107.000,42100.000000,65180.00000,170069.000


In [25]:
segs[segs.LANE_WIDTH > 12]

,LNSegID,ROUTE_ID,BEGIN_MP,END_MP,FS_SYSTEM,RURAL_URBAN,AT_GRADE_OTHER,AT_GRADE_SIGNALS,AT_GRADE_SIGNS,DegHCurve_LWA,DegGrade_LWA,AADT,DirAADT,PEAK_CAPACITY_HCM6,TRUCK_PERC,TYPE_TERRAIN,THROUGH_LANES,LANE_WIDTH,SHLD_WIDTH_L,SHLD_WIDTH_R,SPEED_LIMIT_LWA,MEDIAN_TYPE,MEDIAN_WIDTH,SECTION_LENGTH,INTERCHANGE_SEG
767,768,024-EB-9004 -000,0.0,0.299,2,Rural,0,0,0,1.7,2.540301,12481,6240.0,3955.0,25.911,1.0,2.0,15.0,6.0,10.0,70.0,2.0,99.0,0.299,Yes


### Split data into k_folds

In [3]:
def data_split(numsplit, dataframe, unique_segID):
    df = dataframe.copy()
    from math import ceil, floor
    cols = list(df.columns)
    cols.append('split')
    data = pd.DataFrame(columns = cols) 
    
    num_samples = ceil(len(df[unique_segID].unique())/numsplit)
    
    for i in range(numsplit):
        if i < (numsplit-1):
            selection = df.sample(n = num_samples, random_state = 42)
            selection['split'] = i+1
            data = pd.concat([data,selection], ignore_index=True)
        
            df = df.loc[~(df[unique_segID].isin(selection[unique_segID]))]
        
        else:
            selection = df
            selection['split'] = i+1
            data = pd.concat([data,selection], ignore_index=True)
            
    return data

In [4]:
cdu  = data_split(6, segs, 'LNSegID')
cdu = cdu.sort_values(['LNSegID'], ascending=True)
cdu.head()

,LNSegID,ROUTE_ID,BEGIN_MP,END_MP,FS_SYSTEM,RURAL_URBAN,AT_GRADE_OTHER,AT_GRADE_SIGNALS,AT_GRADE_SIGNS,DegHCurve_LWA,DegGrade_LWA,AADT,DirAADT,PEAK_CAPACITY_HCM6,TRUCK_PERC,TYPE_TERRAIN,THROUGH_LANES,LANE_WIDTH,SHLD_WIDTH_L,SHLD_WIDTH_R,SPEED_LIMIT_LWA,MEDIAN_TYPE,MEDIAN_WIDTH,SECTION_LENGTH,INTERCHANGE_SEG,split
993,1,001-LN-9008 -000,36.401,46.302,2,Rural,0,0,0,1.7,2.608974,4386,2193.0,1805.0,27.247,2.0,4.0,12.0,6.0,10.0,70.0,2.0,36.0,9.901,No,5
1205,2,001-LN-9008 -000,46.942,48.537,2,Rural,0,0,0,1.7,1.450000,8643,4322.0,1652.0,25.012,2.0,4.0,12.0,6.0,10.0,70.0,2.0,36.0,1.595,No,6
690,3,001-LN-9008 -000,49.183,57.552,2,Rural,0,0,0,1.7,1.450000,7953,3976.0,1593.0,25.012,2.0,4.0,12.0,6.0,10.0,70.0,2.0,36.0,8.369,No,3
830,4,003-BG-9002 -000,45.039,47.596,2,Rural,0,0,0,1.7,1.632245,16235,8118.0,3501.0,13.172,1.0,4.0,12.0,5.0,10.0,70.0,2.0,36.0,2.557,No,4
1206,5,003-BG-9002 -000,48.031,58.609,2,Rural,0,0,0,1.7,2.095735,12370,6185.0,2782.0,28.638,1.0,4.0,12.0,5.0,10.0,70.0,2.0,36.0,10.578,No,6


In [5]:
cdu.groupby('split', as_index=False).agg({'LNSegID':'count'})

,split,LNSegID
0,1,241
1,2,241
2,3,241
3,4,241
4,5,241
5,6,240


### Add crashes by hour

In [6]:
cr = pd.read_parquet('crash_aggregated.parquet')
cr.head()

,LNSegID,Hour,TotalCrashes,TotalKilled,TotalInjury,Total_A,Total_B,Total_C,TotalNonInjury
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
cd = pd.merge(cdu, cr, on='LNSegID', how='left')
print(len(cd))
cd['Length'] = abs(cd['END_MP'] - cd['BEGIN_MP'])
cd.head()

33576


,LNSegID,ROUTE_ID,BEGIN_MP,END_MP,FS_SYSTEM,RURAL_URBAN,AT_GRADE_OTHER,AT_GRADE_SIGNALS,AT_GRADE_SIGNS,DegHCurve_LWA,DegGrade_LWA,AADT,DirAADT,PEAK_CAPACITY_HCM6,TRUCK_PERC,TYPE_TERRAIN,THROUGH_LANES,LANE_WIDTH,SHLD_WIDTH_L,SHLD_WIDTH_R,SPEED_LIMIT_LWA,MEDIAN_TYPE,MEDIAN_WIDTH,SECTION_LENGTH,INTERCHANGE_SEG,split,Hour,TotalCrashes,TotalKilled,TotalInjury,Total_A,Total_B,Total_C,TotalNonInjury,Length
0,1,001-LN-9008 -000,36.401,46.302,2,Rural,0,0,0,1.7,2.608974,4386,2193.0,1805.0,27.247,2.0,4.0,12.0,6.0,10.0,70.0,2.0,36.0,9.901,No,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.901
1,1,001-LN-9008 -000,36.401,46.302,2,Rural,0,0,0,1.7,2.608974,4386,2193.0,1805.0,27.247,2.0,4.0,12.0,6.0,10.0,70.0,2.0,36.0,9.901,No,5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.901
2,1,001-LN-9008 -000,36.401,46.302,2,Rural,0,0,0,1.7,2.608974,4386,2193.0,1805.0,27.247,2.0,4.0,12.0,6.0,10.0,70.0,2.0,36.0,9.901,No,5,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.901
3,1,001-LN-9008 -000,36.401,46.302,2,Rural,0,0,0,1.7,2.608974,4386,2193.0,1805.0,27.247,2.0,4.0,12.0,6.0,10.0,70.0,2.0,36.0,9.901,No,5,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.901
4,1,001-LN-9008 -000,36.401,46.302,2,Rural,0,0,0,1.7,2.608974,4386,2193.0,1805.0,27.247,2.0,4.0,12.0,6.0,10.0,70.0,2.0,36.0,9.901,No,5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.901


In [8]:
#convert the new functional system to old functional system
cd['fun_c']=np.where(cd['RURAL_URBAN']=='Rural', np.where(cd['FS_SYSTEM']==1, 1, np.where(cd['FS_SYSTEM']==3, 2, 
                            np.where(cd['FS_SYSTEM']==4, 6, np.where(cd['FS_SYSTEM']==5, 7, np.where(cd['FS_SYSTEM']==6, 8, 9))))),
                            np.where(cd['FS_SYSTEM']==1, 11, np.where(cd['FS_SYSTEM']==2, 12, 
                            np.where(cd['FS_SYSTEM']==3, 14, np.where(cd['FS_SYSTEM']==4, 16, np.where(cd['FS_SYSTEM']==5, 17, 19))))))

cd.head()

,LNSegID,ROUTE_ID,BEGIN_MP,END_MP,FS_SYSTEM,RURAL_URBAN,AT_GRADE_OTHER,AT_GRADE_SIGNALS,AT_GRADE_SIGNS,DegHCurve_LWA,DegGrade_LWA,AADT,DirAADT,PEAK_CAPACITY_HCM6,TRUCK_PERC,TYPE_TERRAIN,THROUGH_LANES,LANE_WIDTH,SHLD_WIDTH_L,SHLD_WIDTH_R,SPEED_LIMIT_LWA,MEDIAN_TYPE,MEDIAN_WIDTH,SECTION_LENGTH,INTERCHANGE_SEG,split,Hour,TotalCrashes,TotalKilled,TotalInjury,Total_A,Total_B,Total_C,TotalNonInjury,Length,fun_c
0,1,001-LN-9008 -000,36.401,46.302,2,Rural,0,0,0,1.7,2.608974,4386,2193.0,1805.0,27.247,2.0,4.0,12.0,6.0,10.0,70.0,2.0,36.0,9.901,No,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.901,9
1,1,001-LN-9008 -000,36.401,46.302,2,Rural,0,0,0,1.7,2.608974,4386,2193.0,1805.0,27.247,2.0,4.0,12.0,6.0,10.0,70.0,2.0,36.0,9.901,No,5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.901,9
2,1,001-LN-9008 -000,36.401,46.302,2,Rural,0,0,0,1.7,2.608974,4386,2193.0,1805.0,27.247,2.0,4.0,12.0,6.0,10.0,70.0,2.0,36.0,9.901,No,5,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.901,9
3,1,001-LN-9008 -000,36.401,46.302,2,Rural,0,0,0,1.7,2.608974,4386,2193.0,1805.0,27.247,2.0,4.0,12.0,6.0,10.0,70.0,2.0,36.0,9.901,No,5,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.901,9
4,1,001-LN-9008 -000,36.401,46.302,2,Rural,0,0,0,1.7,2.608974,4386,2193.0,1805.0,27.247,2.0,4.0,12.0,6.0,10.0,70.0,2.0,36.0,9.901,No,5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.901,9


In [9]:
cd['MEDIAN_TYPE'] = cd['MEDIAN_TYPE'].astype('category')
cd['INTERCHANGE_SEG'] = cd['INTERCHANGE_SEG'].astype('category')
cd['TYPE_TERRAIN'] = cd['TYPE_TERRAIN'].astype('category')
cd['RURAL_URBAN'] = cd['RURAL_URBAN'].astype('category')
cd.head()

,LNSegID,ROUTE_ID,BEGIN_MP,END_MP,FS_SYSTEM,RURAL_URBAN,AT_GRADE_OTHER,AT_GRADE_SIGNALS,AT_GRADE_SIGNS,DegHCurve_LWA,DegGrade_LWA,AADT,DirAADT,PEAK_CAPACITY_HCM6,TRUCK_PERC,TYPE_TERRAIN,THROUGH_LANES,LANE_WIDTH,SHLD_WIDTH_L,SHLD_WIDTH_R,SPEED_LIMIT_LWA,MEDIAN_TYPE,MEDIAN_WIDTH,SECTION_LENGTH,INTERCHANGE_SEG,split,Hour,TotalCrashes,TotalKilled,TotalInjury,Total_A,Total_B,Total_C,TotalNonInjury,Length,fun_c
0,1,001-LN-9008 -000,36.401,46.302,2,Rural,0,0,0,1.7,2.608974,4386,2193.0,1805.0,27.247,2.0,4.0,12.0,6.0,10.0,70.0,2.0,36.0,9.901,No,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.901,9
1,1,001-LN-9008 -000,36.401,46.302,2,Rural,0,0,0,1.7,2.608974,4386,2193.0,1805.0,27.247,2.0,4.0,12.0,6.0,10.0,70.0,2.0,36.0,9.901,No,5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.901,9
2,1,001-LN-9008 -000,36.401,46.302,2,Rural,0,0,0,1.7,2.608974,4386,2193.0,1805.0,27.247,2.0,4.0,12.0,6.0,10.0,70.0,2.0,36.0,9.901,No,5,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.901,9
3,1,001-LN-9008 -000,36.401,46.302,2,Rural,0,0,0,1.7,2.608974,4386,2193.0,1805.0,27.247,2.0,4.0,12.0,6.0,10.0,70.0,2.0,36.0,9.901,No,5,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.901,9
4,1,001-LN-9008 -000,36.401,46.302,2,Rural,0,0,0,1.7,2.608974,4386,2193.0,1805.0,27.247,2.0,4.0,12.0,6.0,10.0,70.0,2.0,36.0,9.901,No,5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.901,9


In [10]:
k_weight = 1659000/12200
a_weight = 96200/12200
b_weight = 27800/12200
c_weight = 22800/12200

cd['TotalCrashes'] = cd['TotalCrashes'].fillna(0)
cd['TotalKilled'] = cd['TotalKilled'].fillna(0)
cd['TotalInjury'] = cd['TotalInjury'].fillna(0)
cd['TotalNonInjury'] = cd['TotalNonInjury'].fillna(0)
cd['Total_A'] = cd['Total_A'].fillna(0)
cd['Total_B'] = cd['Total_B'].fillna(0)
cd['Total_C'] = cd['Total_C'].fillna(0)

cd['EPDO'] = (cd['TotalKilled'] * k_weight) + (cd['Total_A'] * a_weight) + (cd['Total_B'] * b_weight) + \
                  (cd['Total_C'] * c_weight) +   (cd['TotalNonInjury'])

cd.head()

,LNSegID,ROUTE_ID,BEGIN_MP,END_MP,FS_SYSTEM,RURAL_URBAN,AT_GRADE_OTHER,AT_GRADE_SIGNALS,AT_GRADE_SIGNS,DegHCurve_LWA,DegGrade_LWA,AADT,DirAADT,PEAK_CAPACITY_HCM6,TRUCK_PERC,TYPE_TERRAIN,THROUGH_LANES,LANE_WIDTH,SHLD_WIDTH_L,SHLD_WIDTH_R,SPEED_LIMIT_LWA,MEDIAN_TYPE,MEDIAN_WIDTH,SECTION_LENGTH,INTERCHANGE_SEG,split,Hour,TotalCrashes,TotalKilled,TotalInjury,Total_A,Total_B,Total_C,TotalNonInjury,Length,fun_c,EPDO
0,1,001-LN-9008 -000,36.401,46.302,2,Rural,0,0,0,1.7,2.608974,4386,2193.0,1805.0,27.247,2.0,4.0,12.0,6.0,10.0,70.0,2.0,36.0,9.901,No,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.901,9,0.0
1,1,001-LN-9008 -000,36.401,46.302,2,Rural,0,0,0,1.7,2.608974,4386,2193.0,1805.0,27.247,2.0,4.0,12.0,6.0,10.0,70.0,2.0,36.0,9.901,No,5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.901,9,0.0
2,1,001-LN-9008 -000,36.401,46.302,2,Rural,0,0,0,1.7,2.608974,4386,2193.0,1805.0,27.247,2.0,4.0,12.0,6.0,10.0,70.0,2.0,36.0,9.901,No,5,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.901,9,0.0
3,1,001-LN-9008 -000,36.401,46.302,2,Rural,0,0,0,1.7,2.608974,4386,2193.0,1805.0,27.247,2.0,4.0,12.0,6.0,10.0,70.0,2.0,36.0,9.901,No,5,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.901,9,0.0
4,1,001-LN-9008 -000,36.401,46.302,2,Rural,0,0,0,1.7,2.608974,4386,2193.0,1805.0,27.247,2.0,4.0,12.0,6.0,10.0,70.0,2.0,36.0,9.901,No,5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.901,9,0.0


### Speeds

In [11]:
spd18 = pd.read_parquet('aggregated_freeway_speeds_18.parquet')
spd18 = spd18.loc[(spd18.MeanSpeed >= 5) & (spd18.MeanSpeed <= 100)] #filter out < 5mph speeds
spd18['DateTime']=pd.to_datetime(spd18['DateTime'])
spd18['dow'] = spd18['DateTime'].dt.dayofweek
spd18['wkend'] = np.where(spd18['dow']>4, 1, 0)
spd18 = spd18.loc[spd18.wkend == 0] #weekday speeds
print(len(spd18))
spd18.head()

6659756


,LNSegID,DateTime,Hour,MeanTT,SegLength,MeanSpeed,dow,wkend
0,7,2018-01-01 00:00:00,0,0.013602,0.933,68.590613,0,0
1,7,2018-01-01 01:00:00,1,0.015959,0.933,58.462940,0,0
2,7,2018-01-01 02:00:00,2,0.014963,0.933,62.355829,0,0
3,7,2018-01-01 03:00:00,3,0.014686,0.933,63.529961,0,0
4,7,2018-01-01 04:00:00,4,0.014161,0.933,65.886809,0,0


In [12]:
spd19 = pd.read_parquet('aggregated_freeway_speeds_19.parquet')
spd19 = spd19.loc[(spd19.MeanSpeed >= 5) & (spd19.MeanSpeed <= 100)] #filter out < 5mph speeds
spd19['DateTime']=pd.to_datetime(spd19['DateTime'])
spd19['dow'] = spd19['DateTime'].dt.dayofweek
spd19['wkend'] = np.where(spd19['dow']>4, 1, 0)
spd19 = spd19.loc[spd19.wkend == 0] #weekday speeds
print(len(spd19))
spd19.head()

6688746


,LNSegID,DateTime,Hour,MeanTT,SegLength,MeanSpeed,dow,wkend
0,7,2019-01-01 00:00:00,0,0.013781,0.933000,67.703342,1,0
1,7,2019-01-01 01:00:00,1,0.013483,0.933000,69.196092,1,0
2,7,2019-01-01 02:00:00,2,0.014162,0.933000,65.880000,1,0
3,7,2019-01-01 03:00:00,3,0.014085,0.847114,60.144100,1,0
4,7,2019-01-01 04:00:00,4,0.013309,0.933000,70.103310,1,0


In [13]:
spd = pd.concat([spd18, spd19], ignore_index=False)
print(len(spd))
spd.head()

13348502


,LNSegID,DateTime,Hour,MeanTT,SegLength,MeanSpeed,dow,wkend
0,7,2018-01-01 00:00:00,0,0.013602,0.933,68.590613,0,0
1,7,2018-01-01 01:00:00,1,0.015959,0.933,58.462940,0,0
2,7,2018-01-01 02:00:00,2,0.014963,0.933,62.355829,0,0
3,7,2018-01-01 03:00:00,3,0.014686,0.933,63.529961,0,0
4,7,2018-01-01 04:00:00,4,0.014161,0.933,65.886809,0,0


In [14]:
spd[['Hour','MeanTT','MeanSpeed']].describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Hour,13348502.0,11.548635,6.844920,0.000000e+00,6.000000,12.000000,17.000000,23.000000
MeanTT,13348502.0,0.012872,0.021857,1.758769e-16,0.003393,0.004087,0.011108,1.247748
MeanSpeed,13348502.0,66.252140,6.615695,5.001083e+00,64.105265,68.100526,70.157941,99.750000


In [15]:
def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

In [16]:
spd_pcnt = spd.groupby(['LNSegID','Hour'], as_index=False).agg({'MeanSpeed':[percentile(5),percentile(10),percentile(15),
                                                                   percentile(20), percentile(25), percentile(30),
                                                                   percentile(35), percentile(40), percentile(45),
                                                                   percentile(50), percentile(55), percentile(60), 
                                                                   percentile(65), percentile(70), percentile(75), 
                                                                   percentile(80), percentile(85), percentile(90), 
                                                                   percentile(95), percentile(99)]})

spd_pcnt.columns = ['LNSegID','Hour','pcnt5spd', 'pcnt10spd', 'pcnt15spd', 'pcnt20spd', 'pcnt25spd','pcnt30spd', 
                    'pcnt35spd','pcnt40spd', 'pcnt45spd', 'pcnt50spd', 'pcnt55spd', 'pcnt60spd', 'pcnt65spd', 
                    'pcnt70spd', 'pcnt75spd','pcnt80spd', 'pcnt85spd', 'pcnt90spd', 'pcnt95spd', 'pcnt99spd',]
spd_pcnt.head()

,LNSegID,Hour,pcnt5spd,pcnt10spd,pcnt15spd,pcnt20spd,pcnt25spd,pcnt30spd,pcnt35spd,pcnt40spd,pcnt45spd,pcnt50spd,pcnt55spd,pcnt60spd,pcnt65spd,pcnt70spd,pcnt75spd,pcnt80spd,pcnt85spd,pcnt90spd,pcnt95spd,pcnt99spd
0,7,0,64.070657,65.044806,65.576420,65.888303,66.174436,66.468505,66.693519,66.945764,67.085446,67.346371,67.585569,67.772278,68.038487,68.404686,68.674880,69.008475,69.405251,69.876739,70.391215,71.719605
1,7,1,63.889741,65.037625,65.616478,66.012151,66.422118,66.734785,67.061493,67.295773,67.498176,67.706086,67.935032,68.186622,68.420882,68.626370,68.781604,69.128588,69.519635,69.936358,70.681713,71.672838
2,7,2,64.336285,65.598324,66.109249,66.460661,66.730345,66.946567,67.207078,67.441540,67.634956,67.823273,68.034824,68.235127,68.452780,68.765326,69.053600,69.292055,69.599681,70.123466,70.691512,72.055428
3,7,3,64.112010,65.249274,66.031336,66.418353,66.850313,67.214298,67.470215,67.734318,68.022178,68.198540,68.470056,68.658057,68.822144,69.031421,69.414546,69.683854,69.988397,70.451633,71.343983,73.159486
4,7,4,65.249049,65.954651,66.634018,67.135308,67.426241,67.617997,67.925149,68.181478,68.345116,68.529359,68.801663,69.024749,69.183073,69.422093,69.652466,69.964686,70.219135,70.639503,71.325300,72.484373


### Merge Speed and HIS

In [18]:
cds = pd.merge(cd, spd, on=['LNSegID','Hour'], how='inner')
print(len(cds))
cds.head()

13348502


,LNSegID,ROUTE_ID,BEGIN_MP,END_MP,FS_SYSTEM,RURAL_URBAN,AT_GRADE_OTHER,AT_GRADE_SIGNALS,AT_GRADE_SIGNS,DegHCurve_LWA,DegGrade_LWA,AADT,DirAADT,PEAK_CAPACITY_HCM6,TRUCK_PERC,TYPE_TERRAIN,THROUGH_LANES,LANE_WIDTH,SHLD_WIDTH_L,SHLD_WIDTH_R,SPEED_LIMIT_LWA,MEDIAN_TYPE,MEDIAN_WIDTH,SECTION_LENGTH,INTERCHANGE_SEG,split,Hour,TotalCrashes,TotalKilled,TotalInjury,Total_A,Total_B,Total_C,TotalNonInjury,Length,fun_c,EPDO,DateTime,MeanTT,SegLength,MeanSpeed,dow,wkend
0,7,005-I -0065 -000,46.168,47.101,1,Rural,0,0,0,1.7,3.010557,37164,18582.0,2223.0,27.538,2.0,6.0,12.0,14.0,10.0,70.0,7.0,31.0,0.933,No,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.933,1,0.0,2018-01-01,0.013602,0.933,68.590613,0,0
1,7,005-I -0065 -000,46.168,47.101,1,Rural,0,0,0,1.7,3.010557,37164,18582.0,2223.0,27.538,2.0,6.0,12.0,14.0,10.0,70.0,7.0,31.0,0.933,No,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.933,1,0.0,2018-01-02,0.013268,0.933,70.322067,1,0
2,7,005-I -0065 -000,46.168,47.101,1,Rural,0,0,0,1.7,3.010557,37164,18582.0,2223.0,27.538,2.0,6.0,12.0,14.0,10.0,70.0,7.0,31.0,0.933,No,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.933,1,0.0,2018-01-03,0.014194,0.933,65.734102,2,0
3,7,005-I -0065 -000,46.168,47.101,1,Rural,0,0,0,1.7,3.010557,37164,18582.0,2223.0,27.538,2.0,6.0,12.0,14.0,10.0,70.0,7.0,31.0,0.933,No,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.933,1,0.0,2018-01-04,0.013937,0.933,66.945242,3,0
4,7,005-I -0065 -000,46.168,47.101,1,Rural,0,0,0,1.7,3.010557,37164,18582.0,2223.0,27.538,2.0,6.0,12.0,14.0,10.0,70.0,7.0,31.0,0.933,No,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.933,1,0.0,2018-01-05,0.013588,0.933,68.661796,4,0


In [20]:
#hourly factors
hour_factor_new=pd.read_csv('Z:/SHIFT/HERS-ST/HERS Speed Model/ATR_HourlyFactors_2014_2015_2016.csv')
hour_factor_new['Hour']=hour_factor_new['h_hour']-1
hour_factor_new['wkend']=hour_factor_new['dayofweek']
hour_factor_new=hour_factor_new[['fun_c', 'wkend','Hour', 'perc']]

hour_factor_new.head()

,fun_c,wkend,Hour,perc
0,1,0,0,0.0135
1,1,0,1,0.0108
2,1,0,2,0.0099
3,1,0,3,0.0107
4,1,0,4,0.0147


In [22]:
#weekend distribution different from weekday distribution
cdsm = pd.merge(cds, hour_factor_new, on=['fun_c', 'wkend', 'Hour'], how='left')
cdsm['DirHrVol'] = cdsm['DirAADT'] * cdsm['perc'] 
print(len(cdsm))
cdsm.head()

13348502


,LNSegID,ROUTE_ID,BEGIN_MP,END_MP,FS_SYSTEM,RURAL_URBAN,AT_GRADE_OTHER,AT_GRADE_SIGNALS,AT_GRADE_SIGNS,DegHCurve_LWA,DegGrade_LWA,AADT,DirAADT,PEAK_CAPACITY_HCM6,TRUCK_PERC,TYPE_TERRAIN,THROUGH_LANES,LANE_WIDTH,SHLD_WIDTH_L,SHLD_WIDTH_R,SPEED_LIMIT_LWA,MEDIAN_TYPE,MEDIAN_WIDTH,SECTION_LENGTH,INTERCHANGE_SEG,split,Hour,TotalCrashes,TotalKilled,TotalInjury,Total_A,Total_B,Total_C,TotalNonInjury,Length,fun_c,EPDO,DateTime,MeanTT,SegLength,MeanSpeed,dow,wkend,perc,DirHrVol
0,7,005-I -0065 -000,46.168,47.101,1,Rural,0,0,0,1.7,3.010557,37164,18582.0,2223.0,27.538,2.0,6.0,12.0,14.0,10.0,70.0,7.0,31.0,0.933,No,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.933,1,0.0,2018-01-01,0.013602,0.933,68.590613,0,0,0.0135,250.857
1,7,005-I -0065 -000,46.168,47.101,1,Rural,0,0,0,1.7,3.010557,37164,18582.0,2223.0,27.538,2.0,6.0,12.0,14.0,10.0,70.0,7.0,31.0,0.933,No,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.933,1,0.0,2018-01-02,0.013268,0.933,70.322067,1,0,0.0135,250.857
2,7,005-I -0065 -000,46.168,47.101,1,Rural,0,0,0,1.7,3.010557,37164,18582.0,2223.0,27.538,2.0,6.0,12.0,14.0,10.0,70.0,7.0,31.0,0.933,No,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.933,1,0.0,2018-01-03,0.014194,0.933,65.734102,2,0,0.0135,250.857
3,7,005-I -0065 -000,46.168,47.101,1,Rural,0,0,0,1.7,3.010557,37164,18582.0,2223.0,27.538,2.0,6.0,12.0,14.0,10.0,70.0,7.0,31.0,0.933,No,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.933,1,0.0,2018-01-04,0.013937,0.933,66.945242,3,0,0.0135,250.857
4,7,005-I -0065 -000,46.168,47.101,1,Rural,0,0,0,1.7,3.010557,37164,18582.0,2223.0,27.538,2.0,6.0,12.0,14.0,10.0,70.0,7.0,31.0,0.933,No,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.933,1,0.0,2018-01-05,0.013588,0.933,68.661796,4,0,0.0135,250.857


In [23]:
cdsm = cdsm[~pd.isnull(cdsm.MeanSpeed)]
print(len(cdsm))
cdsm.head()

13348502


,LNSegID,ROUTE_ID,BEGIN_MP,END_MP,FS_SYSTEM,RURAL_URBAN,AT_GRADE_OTHER,AT_GRADE_SIGNALS,AT_GRADE_SIGNS,DegHCurve_LWA,DegGrade_LWA,AADT,DirAADT,PEAK_CAPACITY_HCM6,TRUCK_PERC,TYPE_TERRAIN,THROUGH_LANES,LANE_WIDTH,SHLD_WIDTH_L,SHLD_WIDTH_R,SPEED_LIMIT_LWA,MEDIAN_TYPE,MEDIAN_WIDTH,SECTION_LENGTH,INTERCHANGE_SEG,split,Hour,TotalCrashes,TotalKilled,TotalInjury,Total_A,Total_B,Total_C,TotalNonInjury,Length,fun_c,EPDO,DateTime,MeanTT,SegLength,MeanSpeed,dow,wkend,perc,DirHrVol
0,7,005-I -0065 -000,46.168,47.101,1,Rural,0,0,0,1.7,3.010557,37164,18582.0,2223.0,27.538,2.0,6.0,12.0,14.0,10.0,70.0,7.0,31.0,0.933,No,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.933,1,0.0,2018-01-01,0.013602,0.933,68.590613,0,0,0.0135,250.857
1,7,005-I -0065 -000,46.168,47.101,1,Rural,0,0,0,1.7,3.010557,37164,18582.0,2223.0,27.538,2.0,6.0,12.0,14.0,10.0,70.0,7.0,31.0,0.933,No,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.933,1,0.0,2018-01-02,0.013268,0.933,70.322067,1,0,0.0135,250.857
2,7,005-I -0065 -000,46.168,47.101,1,Rural,0,0,0,1.7,3.010557,37164,18582.0,2223.0,27.538,2.0,6.0,12.0,14.0,10.0,70.0,7.0,31.0,0.933,No,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.933,1,0.0,2018-01-03,0.014194,0.933,65.734102,2,0,0.0135,250.857
3,7,005-I -0065 -000,46.168,47.101,1,Rural,0,0,0,1.7,3.010557,37164,18582.0,2223.0,27.538,2.0,6.0,12.0,14.0,10.0,70.0,7.0,31.0,0.933,No,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.933,1,0.0,2018-01-04,0.013937,0.933,66.945242,3,0,0.0135,250.857
4,7,005-I -0065 -000,46.168,47.101,1,Rural,0,0,0,1.7,3.010557,37164,18582.0,2223.0,27.538,2.0,6.0,12.0,14.0,10.0,70.0,7.0,31.0,0.933,No,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.933,1,0.0,2018-01-05,0.013588,0.933,68.661796,4,0,0.0135,250.857


In [26]:
cdd = cdsm[cdsm.LANE_WIDTH <= 12]

In [41]:
cdd['FS_SYSTEM'] = cdd['FS_SYSTEM'].astype('int')
cdd['MEDIAN_TYPE'] = cdd['MEDIAN_TYPE'].astype('category')
cdd['INTERCHANGE_SEG'] = cdd['INTERCHANGE_SEG'].astype('category')
cdd['TYPE_TERRAIN'] = cdd['TYPE_TERRAIN'].astype('category')
cdd['RURAL_URBAN'] = cdd['RURAL_URBAN'].astype('category')
cdd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13348502 entries, 0 to 13348501
Data columns (total 45 columns):
 #   Column              Dtype         
---  ------              -----         
 0   LNSegID             object        
 1   ROUTE_ID            object        
 2   BEGIN_MP            float64       
 3   END_MP              float64       
 4   FS_SYSTEM           int32         
 5   RURAL_URBAN         category      
 6   AT_GRADE_OTHER      object        
 7   AT_GRADE_SIGNALS    object        
 8   AT_GRADE_SIGNS      object        
 9   DegHCurve_LWA       float64       
 10  DegGrade_LWA        float64       
 11  AADT                object        
 12  DirAADT             float64       
 13  PEAK_CAPACITY_HCM6  float64       
 14  TRUCK_PERC          float64       
 15  TYPE_TERRAIN        category      
 16  THROUGH_LANES       float64       
 17  LANE_WIDTH          float64       
 18  SHLD_WIDTH_L        float64       
 19  SHLD_WIDTH_R        float64       
 20  

### Training

In [59]:
var = ['LNSegID', 'Hour', 'Length',  'DirHrVol','RURAL_URBAN','TYPE_TERRAIN','MEDIAN_TYPE','TRUCK_PERC',
       'LANE_WIDTH', 'MEDIAN_WIDTH', 'SHLD_WIDTH_L', 'THROUGH_LANES', 'FS_SYSTEM', 'INTERCHANGE_SEG',
     'SHLD_WIDTH_R','SPEED_LIMIT_LWA', 'PEAK_CAPACITY_HCM6', 'EPDO', ] #DegHCurve_LWA, DegGrade_LWA 

In [60]:
folds = sorted(list(cdsm.split.unique()))
test = folds.pop()
folds

[1, 2, 3, 4, 5]

In [61]:
test

6

In [67]:
#Expectile
xgblss = XGBoostLSS(
    Expectile(stabilization="L2",              # Options are "None", "MAD", "L2".
              expectiles = [0.05, 0.10, 0.15, 0.20, 0.25, 0.30, 0.35, 0.40, 0.45, 0.50, 0.55, 0.60, 0.65, 0.70, 0.75, 0.80, 0.85, 0.90, 0.95, 0.99],         # List of expectiles to be estimated, in increasing order.
              penalize_crossing = True           # Whether to include a penalty term to discourage crossing of expectiles.
              )    
)

In [68]:
train_rmse5 =  []
train_rmse10 = []
train_rmse15 = []
train_rmse20 = []
train_rmse25 = []
train_rmse30 = []
train_rmse35 = []
train_rmse40 = []
train_rmse45 = []
train_rmse50 = []
train_rmse55 = []
train_rmse60 = []
train_rmse65 = []
train_rmse70 = []
train_rmse75 = []
train_rmse80 = []
train_rmse85 = []
train_rmse90 = []
train_rmse95 = []
train_rmse99 = []

In [69]:
train_mae5 =  []
train_mae10 = []
train_mae15 = []
train_mae20 = []
train_mae25 = []
train_mae30 = []
train_mae35 = []
train_mae40 = []
train_mae45 = []
train_mae50 = []
train_mae55 = []
train_mae60 = []
train_mae65 = []
train_mae70 = []
train_mae75 = []
train_mae80 = []
train_mae85 = []
train_mae90 = []
train_mae95 = []
train_mae99 = []

In [70]:
train_mape5 =  []
train_mape10 = []
train_mape15 = []
train_mape20 = []
train_mape25 = []
train_mape30 = []
train_mape35 = []
train_mape40 = []
train_mape45 = []
train_mape50 = []
train_mape55 = []
train_mape60 = []
train_mape65 = []
train_mape70 = []
train_mape75 = []
train_mape80 = []
train_mape85 = []
train_mape90 = []
train_mape95 = []
train_mape99 = []

In [71]:
validation_rmse5 =  []
validation_rmse10 = []
validation_rmse15 = []
validation_rmse20 = []
validation_rmse25 = []
validation_rmse30 = []
validation_rmse35 = []
validation_rmse40 = []
validation_rmse45 = []
validation_rmse50 = []
validation_rmse55 = []
validation_rmse60 = []
validation_rmse65 = []
validation_rmse70 = []
validation_rmse75 = []
validation_rmse80 = []
validation_rmse85 = []
validation_rmse90 = []
validation_rmse95 = []
validation_rmse99 = []

In [72]:
validation_mae5 =  []
validation_mae10 = []
validation_mae15 = []
validation_mae20 = []
validation_mae25 = []
validation_mae30 = []
validation_mae35 = []
validation_mae40 = []
validation_mae45 = []
validation_mae50 = []
validation_mae55 = []
validation_mae60 = []
validation_mae65 = []
validation_mae70 = []
validation_mae75 = []
validation_mae80 = []
validation_mae85 = []
validation_mae90 = []
validation_mae95 = []
validation_mae99 = []

In [73]:
validation_mape5 =  []
validation_mape10 = []
validation_mape15 = []
validation_mape20 = []
validation_mape25 = []
validation_mape30 = []
validation_mape35 = []
validation_mape40 = []
validation_mape45 = []
validation_mape50 = []
validation_mape55 = []
validation_mape60 = []
validation_mape65 = []
validation_mape70 = []
validation_mape75 = []
validation_mape80 = []
validation_mape85 = []
validation_mape90 = []
validation_mape95 = []
validation_mape99 = []

In [ ]:
import statistics
for k in folds:
    
    #training set
    dt = cdd.loc[cdd.split != k]
    x = dt[var]
    y = dt[['MeanSpeed']]

    #validation set
    vd = cdd.loc[cdd.split == k]
    vx = vd[var]
    vy = vd[['MeanSpeed']]

    #convert to DMatrixes
    dtrain = xgb.DMatrix(x.iloc[:,1:], label=y, enable_categorical=True)
    dvalid = xgb.DMatrix(vx.iloc[:,1:], label=vy, enable_categorical=True)
    
    params = {
    "eta" : 0.09,
    "max_depth" : 6,
    "gamma" : 0.05,
    "subsample": 0.9,
    "colsample_bytree": 1,
    "min_child_weight": 1,
    "booster" : 'gbtree', 
    "tree_method": 'gpu_hist',
    "sampling_method":'gradient_based',
    # "gpu_id":         ["none", [0]]
    }

    # Train Model with optimized hyperparameters
    n_rounds = 5000
    xgblss.train(params,
             dtrain,
             num_boost_round=n_rounds
             )
    
    #train scores
    train_expectile = xgblss.predict(dtrain, pred_type="expectiles")
    xtr = x[['LNSegID','Hour','Length']].join(train_expectile)
    xts = xtr.merge(spd_pcnt, on=['LNSegID','Hour'], how='left') #xts - x train score
    xts = xts.loc[~pd.isnull(xts.pcnt5spd)]
    
    ktrain_rmse5 = round(mse(xts['pcnt5spd'], xts['expectile_0.05'], squared=False),2)
    ktrain_rmse10 = round(mse(xts['pcnt10spd'], xts['expectile_0.1'], squared=False),2)
    ktrain_rmse15 = round(mse(xts['pcnt15spd'], xts['expectile_0.15'], squared=False),2)
    ktrain_rmse20 = round(mse(xts['pcnt20spd'], xts['expectile_0.2'], squared=False),2)
    ktrain_rmse25 = round(mse(xts['pcnt25spd'], xts['expectile_0.25'], squared=False),2)
    ktrain_rmse30 = round(mse(xts['pcnt30spd'], xts['expectile_0.3'], squared=False),2)
    ktrain_rmse35 = round(mse(xts['pcnt35spd'], xts['expectile_0.35'], squared=False),2)
    ktrain_rmse40 = round(mse(xts['pcnt40spd'], xts['expectile_0.4'], squared=False),2)
    ktrain_rmse45 = round(mse(xts['pcnt45spd'], xts['expectile_0.45'], squared=False),2)
    ktrain_rmse50 = round(mse(xts['pcnt50spd'], xts['expectile_0.5'], squared=False),2)
    ktrain_rmse55 = round(mse(xts['pcnt55spd'], xts['expectile_0.55'], squared=False),2)
    ktrain_rmse60 = round(mse(xts['pcnt60spd'], xts['expectile_0.6'], squared=False),2)
    ktrain_rmse65 = round(mse(xts['pcnt65spd'], xts['expectile_0.65'], squared=False),2)
    ktrain_rmse70 = round(mse(xts['pcnt70spd'], xts['expectile_0.7'], squared=False),2)
    ktrain_rmse75 = round(mse(xts['pcnt75spd'], xts['expectile_0.75'], squared=False),2)
    ktrain_rmse80 = round(mse(xts['pcnt80spd'], xts['expectile_0.8'], squared=False),2)
    ktrain_rmse85 = round(mse(xts['pcnt85spd'], xts['expectile_0.85'], squared=False),2)
    ktrain_rmse90 = round(mse(xts['pcnt90spd'], xts['expectile_0.9'], squared=False),2)
    ktrain_rmse95 = round(mse(xts['pcnt95spd'], xts['expectile_0.95'], squared=False),2)
    ktrain_rmse99 = round(mse(xts['pcnt99spd'], xts['expectile_0.99'], squared=False),2)

    ktrain_mae5 = round(mae(xts['pcnt5spd'], xts['expectile_0.05']),2)
    ktrain_mae10 = round(mae(xts['pcnt10spd'], xts['expectile_0.1']),2)
    ktrain_mae15 = round(mae(xts['pcnt15spd'], xts['expectile_0.15']),2)
    ktrain_mae20 = round(mae(xts['pcnt20spd'], xts['expectile_0.2']),2)
    ktrain_mae25 = round(mae(xts['pcnt25spd'], xts['expectile_0.25']),2)
    ktrain_mae30 = round(mae(xts['pcnt30spd'], xts['expectile_0.3']),2)
    ktrain_mae35 = round(mae(xts['pcnt35spd'], xts['expectile_0.35']),2)
    ktrain_mae40 = round(mae(xts['pcnt40spd'], xts['expectile_0.4']),2)
    ktrain_mae45 = round(mae(xts['pcnt45spd'], xts['expectile_0.45']),2)
    ktrain_mae50 = round(mae(xts['pcnt50spd'], xts['expectile_0.5']),2)
    ktrain_mae55 = round(mae(xts['pcnt55spd'], xts['expectile_0.55']),2)
    ktrain_mae60 = round(mae(xts['pcnt60spd'], xts['expectile_0.6']),2)
    ktrain_mae65 = round(mae(xts['pcnt65spd'], xts['expectile_0.65']),2)
    ktrain_mae70 = round(mae(xts['pcnt70spd'], xts['expectile_0.7']),2)
    ktrain_mae75 = round(mae(xts['pcnt75spd'], xts['expectile_0.75']),2)
    ktrain_mae80 = round(mae(xts['pcnt80spd'], xts['expectile_0.8']),2)
    ktrain_mae85 = round(mae(xts['pcnt85spd'], xts['expectile_0.85']),2)
    ktrain_mae90 = round(mae(xts['pcnt90spd'], xts['expectile_0.9']),2)
    ktrain_mae95 = round(mae(xts['pcnt95spd'], xts['expectile_0.95']),2)
    ktrain_mae99 = round(mae(xts['pcnt99spd'], xts['expectile_0.99']),2)

    ktrain_mape5 = round(mape(xts['pcnt5spd'], xts['expectile_0.05']) * 100,2)
    ktrain_mape10 = round(mape(xts['pcnt10spd'], xts['expectile_0.1']) * 100,2)
    ktrain_mape15 = round(mape(xts['pcnt15spd'], xts['expectile_0.15']) * 100,2)
    ktrain_mape20 = round(mape(xts['pcnt20spd'], xts['expectile_0.2']) * 100,2)
    ktrain_mape25 = round(mape(xts['pcnt25spd'], xts['expectile_0.25']) * 100,2)
    ktrain_mape30 = round(mape(xts['pcnt30spd'], xts['expectile_0.3']) * 100,2)
    ktrain_mape35 = round(mape(xts['pcnt35spd'], xts['expectile_0.35']) * 100,2)
    ktrain_mape40 = round(mape(xts['pcnt40spd'], xts['expectile_0.4']) * 100,2)
    ktrain_mape45 = round(mape(xts['pcnt45spd'], xts['expectile_0.45']) * 100,2)
    ktrain_mape50 = round(mape(xts['pcnt50spd'], xts['expectile_0.5']) * 100,2)
    ktrain_mape55 = round(mape(xts['pcnt55spd'], xts['expectile_0.55']) * 100,2)
    ktrain_mape60 = round(mape(xts['pcnt60spd'], xts['expectile_0.6']) * 100,2)
    ktrain_mape65 = round(mape(xts['pcnt65spd'], xts['expectile_0.65']) * 100,2)
    ktrain_mape70 = round(mape(xts['pcnt70spd'], xts['expectile_0.7']) * 100,2)
    ktrain_mape75 = round(mape(xts['pcnt75spd'], xts['expectile_0.75']) * 100,2)
    ktrain_mape80 = round(mape(xts['pcnt80spd'], xts['expectile_0.8']) * 100,2)
    ktrain_mape85 = round(mape(xts['pcnt85spd'], xts['expectile_0.85']) * 100,2)
    ktrain_mape90 = round(mape(xts['pcnt90spd'], xts['expectile_0.9']) * 100,2)
    ktrain_mape95 = round(mape(xts['pcnt95spd'], xts['expectile_0.95']) * 100,2)
    ktrain_mape99 = round(mape(xts['pcnt99spd'], xts['expectile_0.99']) * 100,2)

    train_rmse5.append(ktrain_rmse5)
    train_rmse10.append(ktrain_rmse10)
    train_rmse15.append(ktrain_rmse15)
    train_rmse20.append(ktrain_rmse20)
    train_rmse25.append(ktrain_rmse25)
    train_rmse30.append(ktrain_rmse30)
    train_rmse35.append(ktrain_rmse35)
    train_rmse40.append(ktrain_rmse40)
    train_rmse45.append(ktrain_rmse45)
    train_rmse50.append(ktrain_rmse50)
    train_rmse55.append(ktrain_rmse55)
    train_rmse60.append(ktrain_rmse60)
    train_rmse65.append(ktrain_rmse65)
    train_rmse70.append(ktrain_rmse70)
    train_rmse75.append(ktrain_rmse75)
    train_rmse80.append(ktrain_rmse80)
    train_rmse85.append(ktrain_rmse85)
    train_rmse90.append(ktrain_rmse90)
    train_rmse95.append(ktrain_rmse95)
    train_rmse99.append(ktrain_rmse99)

    train_mae5.append(ktrain_mae5)
    train_mae10.append(ktrain_mae10)
    train_mae15.append(ktrain_mae15)
    train_mae20.append(ktrain_mae20)
    train_mae25.append(ktrain_mae25)
    train_mae30.append(ktrain_mae30)
    train_mae35.append(ktrain_mae35)
    train_mae40.append(ktrain_mae40)
    train_mae45.append(ktrain_mae45)
    train_mae50.append(ktrain_mae50)
    train_mae55.append(ktrain_mae55)
    train_mae60.append(ktrain_mae60)
    train_mae65.append(ktrain_mae65)
    train_mae70.append(ktrain_mae70)
    train_mae75.append(ktrain_mae75)
    train_mae80.append(ktrain_mae80)
    train_mae85.append(ktrain_mae85)
    train_mae90.append(ktrain_mae90)
    train_mae95.append(ktrain_mae95)
    train_mae99.append(ktrain_mae99)

    train_mape5.append(ktrain_mape5)
    train_mape10.append(ktrain_mape10)
    train_mape15.append(ktrain_mape15)
    train_mape20.append(ktrain_mape20)
    train_mape25.append(ktrain_mape25)
    train_mape30.append(ktrain_mape30)
    train_mape35.append(ktrain_mape35)
    train_mape40.append(ktrain_mape40)
    train_mape45.append(ktrain_mape45)
    train_mape50.append(ktrain_mape50)
    train_mape55.append(ktrain_mape55)
    train_mape60.append(ktrain_mape60)
    train_mape65.append(ktrain_mape65)
    train_mape70.append(ktrain_mape70)
    train_mape75.append(ktrain_mape75)
    train_mape80.append(ktrain_mape80)
    train_mape85.append(ktrain_mape85)
    train_mape90.append(ktrain_mape90)
    train_mape95.append(ktrain_mape95)
    train_mape99.append(ktrain_mape99)


    #validation scores
    validation_expectile = xgblss.predict(dvalid, pred_type="expectiles")
    vxtr = vx[['LNSegID','Hour','Length']].join(validation_expectile)
    vxts = vxtr.merge(spd_pcnt, on=['LNSegID','Hour'], how='left') #vxts - x validation score
    vxts = vxts.loc[~pd.isnull(vxts.pcnt5spd)]
    
    kvalidation_rmse5 = round(mse(vxts['pcnt5spd'], vxts['expectile_0.05'], squared=False),2)
    kvalidation_rmse10 = round(mse(vxts['pcnt10spd'], vxts['expectile_0.1'], squared=False),2)
    kvalidation_rmse15 = round(mse(vxts['pcnt15spd'], vxts['expectile_0.15'], squared=False),2)
    kvalidation_rmse20 = round(mse(vxts['pcnt20spd'], vxts['expectile_0.2'], squared=False),2)
    kvalidation_rmse25 = round(mse(vxts['pcnt25spd'], vxts['expectile_0.25'], squared=False),2)
    kvalidation_rmse30 = round(mse(vxts['pcnt30spd'], vxts['expectile_0.3'], squared=False),2)
    kvalidation_rmse35 = round(mse(vxts['pcnt35spd'], vxts['expectile_0.35'], squared=False),2)
    kvalidation_rmse40 = round(mse(vxts['pcnt40spd'], vxts['expectile_0.4'], squared=False),2)
    kvalidation_rmse45 = round(mse(vxts['pcnt45spd'], vxts['expectile_0.45'], squared=False),2)
    kvalidation_rmse50 = round(mse(vxts['pcnt50spd'], vxts['expectile_0.5'], squared=False),2)
    kvalidation_rmse55 = round(mse(vxts['pcnt55spd'], vxts['expectile_0.55'], squared=False),2)
    kvalidation_rmse60 = round(mse(vxts['pcnt60spd'], vxts['expectile_0.6'], squared=False),2)
    kvalidation_rmse65 = round(mse(vxts['pcnt65spd'], vxts['expectile_0.65'], squared=False),2)
    kvalidation_rmse70 = round(mse(vxts['pcnt70spd'], vxts['expectile_0.7'], squared=False),2)
    kvalidation_rmse75 = round(mse(vxts['pcnt75spd'], vxts['expectile_0.75'], squared=False),2)
    kvalidation_rmse80 = round(mse(vxts['pcnt80spd'], vxts['expectile_0.8'], squared=False),2)
    kvalidation_rmse85 = round(mse(vxts['pcnt85spd'], vxts['expectile_0.85'], squared=False),2)
    kvalidation_rmse90 = round(mse(vxts['pcnt90spd'], vxts['expectile_0.9'], squared=False),2)
    kvalidation_rmse95 = round(mse(vxts['pcnt95spd'], vxts['expectile_0.95'], squared=False),2)
    kvalidation_rmse99 = round(mse(vxts['pcnt99spd'], vxts['expectile_0.99'], squared=False),2)

    kvalidation_mae5 = round(mae(vxts['pcnt5spd'], vxts['expectile_0.05']),2)
    kvalidation_mae10 = round(mae(vxts['pcnt10spd'], vxts['expectile_0.1']),2)
    kvalidation_mae15 = round(mae(vxts['pcnt15spd'], vxts['expectile_0.15']),2)
    kvalidation_mae20 = round(mae(vxts['pcnt20spd'], vxts['expectile_0.2']),2)
    kvalidation_mae25 = round(mae(vxts['pcnt25spd'], vxts['expectile_0.25']),2)
    kvalidation_mae30 = round(mae(vxts['pcnt30spd'], vxts['expectile_0.3']),2)
    kvalidation_mae35 = round(mae(vxts['pcnt35spd'], vxts['expectile_0.35']),2)
    kvalidation_mae40 = round(mae(vxts['pcnt40spd'], vxts['expectile_0.4']),2)
    kvalidation_mae45 = round(mae(vxts['pcnt45spd'], vxts['expectile_0.45']),2)
    kvalidation_mae50 = round(mae(vxts['pcnt50spd'], vxts['expectile_0.5']),2)
    kvalidation_mae55 = round(mae(vxts['pcnt55spd'], vxts['expectile_0.55']),2)
    kvalidation_mae60 = round(mae(vxts['pcnt60spd'], vxts['expectile_0.6']),2)
    kvalidation_mae65 = round(mae(vxts['pcnt65spd'], vxts['expectile_0.65']),2)
    kvalidation_mae70 = round(mae(vxts['pcnt70spd'], vxts['expectile_0.7']),2)
    kvalidation_mae75 = round(mae(vxts['pcnt75spd'], vxts['expectile_0.75']),2)
    kvalidation_mae80 = round(mae(vxts['pcnt80spd'], vxts['expectile_0.8']),2)
    kvalidation_mae85 = round(mae(vxts['pcnt85spd'], vxts['expectile_0.85']),2)
    kvalidation_mae90 = round(mae(vxts['pcnt90spd'], vxts['expectile_0.9']),2)
    kvalidation_mae95 = round(mae(vxts['pcnt95spd'], vxts['expectile_0.95']),2)
    kvalidation_mae99 = round(mae(vxts['pcnt99spd'], vxts['expectile_0.99']),2)

    kvalidation_mape5 = round(mape(vxts['pcnt5spd'], vxts['expectile_0.05']) * 100,2)
    kvalidation_mape10 = round(mape(vxts['pcnt10spd'], vxts['expectile_0.1']) * 100,2)
    kvalidation_mape15 = round(mape(vxts['pcnt15spd'], vxts['expectile_0.15']) * 100,2)
    kvalidation_mape20 = round(mape(vxts['pcnt20spd'], vxts['expectile_0.2']) * 100,2)
    kvalidation_mape25 = round(mape(vxts['pcnt25spd'], vxts['expectile_0.25']) * 100,2)
    kvalidation_mape30 = round(mape(vxts['pcnt30spd'], vxts['expectile_0.3']) * 100,2)
    kvalidation_mape35 = round(mape(vxts['pcnt35spd'], vxts['expectile_0.35']) * 100,2)
    kvalidation_mape40 = round(mape(vxts['pcnt40spd'], vxts['expectile_0.4']) * 100,2)
    kvalidation_mape45 = round(mape(vxts['pcnt45spd'], vxts['expectile_0.45']) * 100,2)
    kvalidation_mape50 = round(mape(vxts['pcnt50spd'], vxts['expectile_0.5']) * 100,2)
    kvalidation_mape55 = round(mape(vxts['pcnt55spd'], vxts['expectile_0.55']) * 100,2)
    kvalidation_mape60 = round(mape(vxts['pcnt60spd'], vxts['expectile_0.6']) * 100,2)
    kvalidation_mape65 = round(mape(vxts['pcnt65spd'], vxts['expectile_0.65']) * 100,2)
    kvalidation_mape70 = round(mape(vxts['pcnt70spd'], vxts['expectile_0.7']) * 100,2)
    kvalidation_mape75 = round(mape(vxts['pcnt75spd'], vxts['expectile_0.75']) * 100,2)
    kvalidation_mape80 = round(mape(vxts['pcnt80spd'], vxts['expectile_0.8']) * 100,2)
    kvalidation_mape85 = round(mape(vxts['pcnt85spd'], vxts['expectile_0.85']) * 100,2)
    kvalidation_mape90 = round(mape(vxts['pcnt90spd'], vxts['expectile_0.9']) * 100,2)
    kvalidation_mape95 = round(mape(vxts['pcnt95spd'], vxts['expectile_0.95']) * 100,2)
    kvalidation_mape99 = round(mape(vxts['pcnt99spd'], vxts['expectile_0.99']) * 100,2)

    validation_rmse5.append(kvalidation_rmse5)
    validation_rmse10.append(kvalidation_rmse10)
    validation_rmse15.append(kvalidation_rmse15)
    validation_rmse20.append(kvalidation_rmse20)
    validation_rmse25.append(kvalidation_rmse25)
    validation_rmse30.append(kvalidation_rmse30)
    validation_rmse35.append(kvalidation_rmse35)
    validation_rmse40.append(kvalidation_rmse40)
    validation_rmse45.append(kvalidation_rmse45)
    validation_rmse50.append(kvalidation_rmse50)
    validation_rmse55.append(kvalidation_rmse55)
    validation_rmse60.append(kvalidation_rmse60)
    validation_rmse65.append(kvalidation_rmse65)
    validation_rmse70.append(kvalidation_rmse70)
    validation_rmse75.append(kvalidation_rmse75)
    validation_rmse80.append(kvalidation_rmse80)
    validation_rmse85.append(kvalidation_rmse85)
    validation_rmse90.append(kvalidation_rmse90)
    validation_rmse95.append(kvalidation_rmse95)
    validation_rmse99.append(kvalidation_rmse99)

    validation_mae5.append(kvalidation_mae5)
    validation_mae10.append(kvalidation_mae10)
    validation_mae15.append(kvalidation_mae15)
    validation_mae20.append(kvalidation_mae20)
    validation_mae25.append(kvalidation_mae25)
    validation_mae30.append(kvalidation_mae30)
    validation_mae35.append(kvalidation_mae35)
    validation_mae40.append(kvalidation_mae40)
    validation_mae45.append(kvalidation_mae45)
    validation_mae50.append(kvalidation_mae50)
    validation_mae55.append(kvalidation_mae55)
    validation_mae60.append(kvalidation_mae60)
    validation_mae65.append(kvalidation_mae65)
    validation_mae70.append(kvalidation_mae70)
    validation_mae75.append(kvalidation_mae75)
    validation_mae80.append(kvalidation_mae80)
    validation_mae85.append(kvalidation_mae85)
    validation_mae90.append(kvalidation_mae90)
    validation_mae95.append(kvalidation_mae95)
    validation_mae99.append(kvalidation_mae99)

    validation_mape5.append(kvalidation_mape5)
    validation_mape10.append(kvalidation_mape10)
    validation_mape15.append(kvalidation_mape15)
    validation_mape20.append(kvalidation_mape20)
    validation_mape25.append(kvalidation_mape25)
    validation_mape30.append(kvalidation_mape30)
    validation_mape35.append(kvalidation_mape35)
    validation_mape40.append(kvalidation_mape40)
    validation_mape45.append(kvalidation_mape45)
    validation_mape50.append(kvalidation_mape50)
    validation_mape55.append(kvalidation_mape55)
    validation_mape60.append(kvalidation_mape60)
    validation_mape65.append(kvalidation_mape65)
    validation_mape70.append(kvalidation_mape70)
    validation_mape75.append(kvalidation_mape75)
    validation_mape80.append(kvalidation_mape80)
    validation_mape85.append(kvalidation_mape85)
    validation_mape90.append(kvalidation_mape90)
    validation_mape95.append(kvalidation_mape95)
    validation_mape99.append(kvalidation_mape99)
    
    print(f'current median training RMSE score: {statistics.mean(train_rmse50)}')
    print(f'current median training MAPE score: {statistics.mean(train_mape50)}')


    print(f'current median Validation RMSE score: {statistics.mean(validation_rmse50)}')
    print(f'current median Validation MAPE score: {statistics.mean(validation_mape50)}')

### Hyperparameter Optimization

In [ ]:
# Any XGBoost hyperparameter can be tuned, where the structure of the parameter dictionary needs to be as follows:

    # Float/Int sample_type
        # {"param_name": ["sample_type", low, high, log]}
            # sample_type: str, Type of sampling, e.g., "float" or "int"
            # low: int, Lower endpoint of the range of suggested values
            # high: int, Upper endpoint of the range of suggested values
            # log: bool, Flag to sample the value from the log domain or not
        # Example: {"eta": "float", low=1e-5, high=1, log=True]}

    # Categorical sample_type
        # {"param_name": ["sample_type", ["choice1", "choice2", "choice3", "..."]]}
            # sample_type: str, Type of sampling, either "categorical"
            # choice1, choice2, choice3, ...: str, Possible choices for the parameter
        # Example: {"booster": ["categorical", ["gbtree", "dart"]]}

    # For parameters without tunable choice (this is needed if tree_method = "gpu_hist" and gpu_id needs to be specified)
        # {"param_name": ["none", [value]]},
            # param_name: str, Name of the parameter
            # value: int, Value of the parameter
        # Example: {"gpu_id": ["none", [0]]}

# Depending on which parameters are optimized, it might happen that some of them are not used, e.g., when {"booster":  ["categorical", ["gbtree", "gblinear"]]} and {"max_depth": ["int", 1, 10, False]} are
# specified, max_depth is not used when gblinear is sampled, since it has no such argument.

param_dict = {
    "eta":              ["float", {"low": 1e-5,   "high": 1,     "log": True}],
    "max_depth":        ["int",   {"low": 1,      "high": 10,    "log": False}],
    "gamma":            ["float", {"low": 1e-8,   "high": 40,    "log": True}],
    "subsample":        ["float", {"low": 0.2,    "high": 1.0,   "log": False}],
    "colsample_bytree": ["float", {"low": 0.2,    "high": 1.0,   "log": False}],
    "booster":          ["categorical", ["gbtree"]],
    # "tree_method":    ["categorical", ["auto", "approx", "hist", "gpu_hist"]],
    # "gpu_id":         ["none", [0]]
}

np.random.seed(123)
opt_param = xgblss.hyper_opt(param_dict,
                             dtrain,
                             num_boost_round=100,        # Number of boosting iterations.
                             nfold=5,                    # Number of cv-folds.
                             early_stopping_rounds=20,   # Number of early-stopping rounds
                             max_minutes=5,              # Time budget in minutes, i.e., stop study after the given number of minutes.
                             n_trials=None,              # The number of trials. If this argument is set to None, there is no limitation on the number of trials.
                             silence=False,              # Controls the verbosity of the trail, i.e., user can silence the outputs of the trail.
                             seed=123,                   # Seed used to generate cv-folds.
                             hp_seed=None                # Seed for random number generator used in the Bayesian hyperparameter search.
                            )

In [ ]:
cols = ['fold', 'pcnt5spd', 'pcnt10spd', 'pcnt15spd', 'pcnt20spd', 'pcnt25spd','pcnt30spd', 
        'pcnt35spd','pcnt40spd', 'pcnt45spd', 'pcnt50spd', 'pcnt55spd', 'pcnt60spd', 'pcnt65spd', 
        'pcnt70spd', 'pcnt75spd','pcnt80spd', 'pcnt85spd', 'pcnt90spd', 'pcnt95spd', 'pcnt99spd',]
train_rmse = pd.DataFrame(columns = cols) 

train_rmse['fold'] = folds
train_rmse['pcnt5spd'] = train_rmse5
train_rmse['pcnt10spd'] = train_rmse10
train_rmse['pcnt15spd'] = train_rmse15
train_rmse['pcnt20spd'] = train_rmse20
train_rmse['pcnt25spd'] = train_rmse25
train_rmse['pcnt30spd'] = train_rmse30
train_rmse['pcnt35spd'] = train_rmse35
train_rmse['pcnt40spd'] = train_rmse40
train_rmse['pcnt45spd'] = train_rmse45
train_rmse['pcnt50spd'] = train_rmse50
train_rmse['pcnt55spd'] = train_rmse55
train_rmse['pcnt60spd'] = train_rmse60
train_rmse['pcnt65spd'] = train_rmse65
train_rmse['pcnt70spd'] = train_rmse70
train_rmse['pcnt75spd'] = train_rmse75
train_rmse['pcnt80spd'] = train_rmse80
train_rmse['pcnt85spd'] = train_rmse85
train_rmse['pcnt90spd'] = train_rmse90
train_rmse['pcnt95spd'] = train_rmse95
train_rmse['pcnt99spd'] = train_rmse99

In [ ]:
cols = ['fold', 'pcnt5spd', 'pcnt10spd', 'pcnt15spd', 'pcnt20spd', 'pcnt25spd','pcnt30spd', 
        'pcnt35spd','pcnt40spd', 'pcnt45spd', 'pcnt50spd', 'pcnt55spd', 'pcnt60spd', 'pcnt65spd', 
        'pcnt70spd', 'pcnt75spd','pcnt80spd', 'pcnt85spd', 'pcnt90spd', 'pcnt95spd', 'pcnt99spd',]
train_mae = pd.DataFrame(columns = cols) 

train_mae['fold'] = folds
train_mae['pcnt5spd'] = train_mae5
train_mae['pcnt10spd'] = train_mae10
train_mae['pcnt15spd'] = train_mae15
train_mae['pcnt20spd'] = train_mae20
train_mae['pcnt25spd'] = train_mae25
train_mae['pcnt30spd'] = train_mae30
train_mae['pcnt35spd'] = train_mae35
train_mae['pcnt40spd'] = train_mae40
train_mae['pcnt45spd'] = train_mae45
train_mae['pcnt50spd'] = train_mae50
train_mae['pcnt55spd'] = train_mae55
train_mae['pcnt60spd'] = train_mae60
train_mae['pcnt65spd'] = train_mae65
train_mae['pcnt70spd'] = train_mae70
train_mae['pcnt75spd'] = train_mae75
train_mae['pcnt80spd'] = train_mae80
train_mae['pcnt85spd'] = train_mae85
train_mae['pcnt90spd'] = train_mae90
train_mae['pcnt95spd'] = train_mae95
train_mae['pcnt99spd'] = train_mae99

In [ ]:
cols = ['fold', 'pcnt5spd', 'pcnt10spd', 'pcnt15spd', 'pcnt20spd', 'pcnt25spd','pcnt30spd', 
        'pcnt35spd','pcnt40spd', 'pcnt45spd', 'pcnt50spd', 'pcnt55spd', 'pcnt60spd', 'pcnt65spd', 
        'pcnt70spd', 'pcnt75spd','pcnt80spd', 'pcnt85spd', 'pcnt90spd', 'pcnt95spd', 'pcnt99spd',]
train_mape = pd.DataFrame(columns = cols) 

train_mape['fold'] = folds
train_mape['pcnt5spd'] = train_mape5
train_mape['pcnt10spd'] = train_mape10
train_mape['pcnt15spd'] = train_mape15
train_mape['pcnt20spd'] = train_mape20
train_mape['pcnt25spd'] = train_mape25
train_mape['pcnt30spd'] = train_mape30
train_mape['pcnt35spd'] = train_mape35
train_mape['pcnt40spd'] = train_mape40
train_mape['pcnt45spd'] = train_mape45
train_mape['pcnt50spd'] = train_mape50
train_mape['pcnt55spd'] = train_mape55
train_mape['pcnt60spd'] = train_mape60
train_mape['pcnt65spd'] = train_mape65
train_mape['pcnt70spd'] = train_mape70
train_mape['pcnt75spd'] = train_mape75
train_mape['pcnt80spd'] = train_mape80
train_mape['pcnt85spd'] = train_mape85
train_mape['pcnt90spd'] = train_mape90
train_mape['pcnt95spd'] = train_mape95
train_mape['pcnt99spd'] = train_mape99

In [ ]:
train_rmse.to_csv('freeway_xgblss_train_rmse.csv', index=False)
train_mae.to_csv('freeway_xgblss_train_mae.csv', index=False)
train_mape.to_csv('freeway_xgblss_train_mape.csv', index=False)

In [ ]:
train_rmse

In [ ]:
train_mae

In [ ]:
train_mape

### Testing

In [ ]:
dt = cdsm.loc[cdsm.split == test]
xt = dt[var]
yt = dt[['MeanSpeed']]
    
dtest = xgb.DMatrix(xt.iloc[:,1:], label=yt, enable_categorical=True)

In [ ]:
#test scores
test_expectile = xgblss.predict(dtest, pred_type="expectiles")
xtr = xt[['LNSegID','Hour','Length']].join(test_expectile)
xts = xtr.merge(spd_pcnt, on=['LNSegID','Hour'], how='left') #xts - x test score
xts = xts.loc[~pd.isnull(xts.pcnt5spd)]

test_rmse5 = round(mse(xts['pcnt5spd'], xts['expectile_0.05'], squared=False),2)
test_rmse10 = round(mse(xts['pcnt10spd'], xts['expectile_0.1'], squared=False),2)
test_rmse15 = round(mse(xts['pcnt15spd'], xts['expectile_0.15'], squared=False),2)
test_rmse20 = round(mse(xts['pcnt20spd'], xts['expectile_0.2'], squared=False),2)
test_rmse25 = round(mse(xts['pcnt25spd'], xts['expectile_0.25'], squared=False),2)
test_rmse30 = round(mse(xts['pcnt30spd'], xts['expectile_0.3'], squared=False),2)
test_rmse35 = round(mse(xts['pcnt35spd'], xts['expectile_0.35'], squared=False),2)
test_rmse40 = round(mse(xts['pcnt40spd'], xts['expectile_0.4'], squared=False),2)
test_rmse45 = round(mse(xts['pcnt45spd'], xts['expectile_0.45'], squared=False),2)
test_rmse50 = round(mse(xts['pcnt50spd'], xts['expectile_0.5'], squared=False),2)
test_rmse55 = round(mse(xts['pcnt55spd'], xts['expectile_0.55'], squared=False),2)
test_rmse60 = round(mse(xts['pcnt60spd'], xts['expectile_0.6'], squared=False),2)
test_rmse65 = round(mse(xts['pcnt65spd'], xts['expectile_0.65'], squared=False),2)
test_rmse70 = round(mse(xts['pcnt70spd'], xts['expectile_0.7'], squared=False),2)
test_rmse75 = round(mse(xts['pcnt75spd'], xts['expectile_0.75'], squared=False),2)
test_rmse80 = round(mse(xts['pcnt80spd'], xts['expectile_0.8'], squared=False),2)
test_rmse85 = round(mse(xts['pcnt85spd'], xts['expectile_0.85'], squared=False),2)
test_rmse90 = round(mse(xts['pcnt90spd'], xts['expectile_0.9'], squared=False),2)
test_rmse95 = round(mse(xts['pcnt95spd'], xts['expectile_0.95'], squared=False),2)
test_rmse99 = round(mse(xts['pcnt99spd'], xts['expectile_0.99'], squared=False),2)

test_mae5 = round(mae(xts['pcnt5spd'], xts['expectile_0.05']),2)
test_mae10 = round(mae(xts['pcnt10spd'], xts['expectile_0.1']),2)
test_mae15 = round(mae(xts['pcnt15spd'], xts['expectile_0.15']),2)
test_mae20 = round(mae(xts['pcnt20spd'], xts['expectile_0.2']),2)
test_mae25 = round(mae(xts['pcnt25spd'], xts['expectile_0.25']),2)
test_mae30 = round(mae(xts['pcnt30spd'], xts['expectile_0.3']),2)
test_mae35 = round(mae(xts['pcnt35spd'], xts['expectile_0.35']),2)
test_mae40 = round(mae(xts['pcnt40spd'], xts['expectile_0.4']),2)
test_mae45 = round(mae(xts['pcnt45spd'], xts['expectile_0.45']),2)
test_mae50 = round(mae(xts['pcnt50spd'], xts['expectile_0.5']),2)
test_mae55 = round(mae(xts['pcnt55spd'], xts['expectile_0.55']),2)
test_mae60 = round(mae(xts['pcnt60spd'], xts['expectile_0.6']),2)
test_mae65 = round(mae(xts['pcnt65spd'], xts['expectile_0.65']),2)
test_mae70 = round(mae(xts['pcnt70spd'], xts['expectile_0.7']),2)
test_mae75 = round(mae(xts['pcnt75spd'], xts['expectile_0.75']),2)
test_mae80 = round(mae(xts['pcnt80spd'], xts['expectile_0.8']),2)
test_mae85 = round(mae(xts['pcnt85spd'], xts['expectile_0.85']),2)
test_mae90 = round(mae(xts['pcnt90spd'], xts['expectile_0.9']),2)
test_mae95 = round(mae(xts['pcnt95spd'], xts['expectile_0.95']),2)
test_mae99 = round(mae(xts['pcnt99spd'], xts['expectile_0.99']),2)

test_mape5 = round(mape(xts['pcnt5spd'], xts['expectile_0.05']) * 100,2)
test_mape10 = round(mape(xts['pcnt10spd'], xts['expectile_0.1']) * 100,2)
test_mape15 = round(mape(xts['pcnt15spd'], xts['expectile_0.15']) * 100,2)
test_mape20 = round(mape(xts['pcnt20spd'], xts['expectile_0.2']) * 100,2)
test_mape25 = round(mape(xts['pcnt25spd'], xts['expectile_0.25']) * 100,2)
test_mape30 = round(mape(xts['pcnt30spd'], xts['expectile_0.3']) * 100,2)
test_mape35 = round(mape(xts['pcnt35spd'], xts['expectile_0.35']) * 100,2)
test_mape40 = round(mape(xts['pcnt40spd'], xts['expectile_0.4']) * 100,2)
test_mape45 = round(mape(xts['pcnt45spd'], xts['expectile_0.45']) * 100,2)
test_mape50 = round(mape(xts['pcnt50spd'], xts['expectile_0.5']) * 100,2)
test_mape55 = round(mape(xts['pcnt55spd'], xts['expectile_0.55']) * 100,2)
test_mape60 = round(mape(xts['pcnt60spd'], xts['expectile_0.6']) * 100,2)
test_mape65 = round(mape(xts['pcnt65spd'], xts['expectile_0.65']) * 100,2)
test_mape70 = round(mape(xts['pcnt70spd'], xts['expectile_0.7']) * 100,2)
test_mape75 = round(mape(xts['pcnt75spd'], xts['expectile_0.75']) * 100,2)
test_mape80 = round(mape(xts['pcnt80spd'], xts['expectile_0.8']) * 100,2)
test_mape85 = round(mape(xts['pcnt85spd'], xts['expectile_0.85']) * 100,2)
test_mape90 = round(mape(xts['pcnt90spd'], xts['expectile_0.9']) * 100,2)
test_mape95 = round(mape(xts['pcnt95spd'], xts['expectile_0.95']) * 100,2)
test_mape99 = round(mape(xts['pcnt99spd'], xts['expectile_0.99']) * 100,2)



print(f'Median testing RMSE score: {test_rmse50}')
print(f'Median testing MAPE score: {test_mape50}')

In [ ]:
cols = ['fold', 'pcnt5spd', 'pcnt10spd', 'pcnt15spd', 'pcnt20spd', 'pcnt25spd','pcnt30spd', 
        'pcnt35spd','pcnt40spd', 'pcnt45spd', 'pcnt50spd', 'pcnt55spd', 'pcnt60spd', 'pcnt65spd', 
        'pcnt70spd', 'pcnt75spd','pcnt80spd', 'pcnt85spd', 'pcnt90spd', 'pcnt95spd', 'pcnt99spd',]
test_rmse = pd.DataFrame(columns = cols) 

test_rmse['Test_fold'] = test
test_rmse['pcnt5spd'] = test_rmse5
test_rmse['pcnt10spd'] = test_rmse10
test_rmse['pcnt15spd'] = test_rmse15
test_rmse['pcnt20spd'] = test_rmse20
test_rmse['pcnt25spd'] = test_rmse25
test_rmse['pcnt30spd'] = test_rmse30
test_rmse['pcnt35spd'] = test_rmse35
test_rmse['pcnt40spd'] = test_rmse40
test_rmse['pcnt45spd'] = test_rmse45
test_rmse['pcnt50spd'] = test_rmse50
test_rmse['pcnt55spd'] = test_rmse55
test_rmse['pcnt60spd'] = test_rmse60
test_rmse['pcnt65spd'] = test_rmse65
test_rmse['pcnt70spd'] = test_rmse70
test_rmse['pcnt75spd'] = test_rmse75
test_rmse['pcnt80spd'] = test_rmse80
test_rmse['pcnt85spd'] = test_rmse85
test_rmse['pcnt90spd'] = test_rmse90
test_rmse['pcnt95spd'] = test_rmse95
test_rmse['pcnt99spd'] = test_rmse99

In [ ]:
cols = ['fold', 'pcnt5spd', 'pcnt10spd', 'pcnt15spd', 'pcnt20spd', 'pcnt25spd','pcnt30spd', 
        'pcnt35spd','pcnt40spd', 'pcnt45spd', 'pcnt50spd', 'pcnt55spd', 'pcnt60spd', 'pcnt65spd', 
        'pcnt70spd', 'pcnt75spd','pcnt80spd', 'pcnt85spd', 'pcnt90spd', 'pcnt95spd', 'pcnt99spd',]
test_mae = pd.DataFrame(columns = cols) 

test_mae['Test_fold'] = test
test_mae['pcnt5spd'] = test_mae5
test_mae['pcnt10spd'] = test_mae10
test_mae['pcnt15spd'] = test_mae15
test_mae['pcnt20spd'] = test_mae20
test_mae['pcnt25spd'] = test_mae25
test_mae['pcnt30spd'] = test_mae30
test_mae['pcnt35spd'] = test_mae35
test_mae['pcnt40spd'] = test_mae40
test_mae['pcnt45spd'] = test_mae45
test_mae['pcnt50spd'] = test_mae50
test_mae['pcnt55spd'] = test_mae55
test_mae['pcnt60spd'] = test_mae60
test_mae['pcnt65spd'] = test_mae65
test_mae['pcnt70spd'] = test_mae70
test_mae['pcnt75spd'] = test_mae75
test_mae['pcnt80spd'] = test_mae80
test_mae['pcnt85spd'] = test_mae85
test_mae['pcnt90spd'] = test_mae90
test_mae['pcnt95spd'] = test_mae95
test_mae['pcnt99spd'] = test_mae99

In [ ]:
cols = ['fold', 'pcnt5spd', 'pcnt10spd', 'pcnt15spd', 'pcnt20spd', 'pcnt25spd','pcnt30spd', 
        'pcnt35spd','pcnt40spd', 'pcnt45spd', 'pcnt50spd', 'pcnt55spd', 'pcnt60spd', 'pcnt65spd', 
        'pcnt70spd', 'pcnt75spd','pcnt80spd', 'pcnt85spd', 'pcnt90spd', 'pcnt95spd', 'pcnt99spd',]
test_mape = pd.DataFrame(columns = cols) 

test_mape['Test_fold'] = test
test_mape['pcnt5spd'] = test_mape5
test_mape['pcnt10spd'] = test_mape10
test_mape['pcnt15spd'] = test_mape15
test_mape['pcnt20spd'] = test_mape20
test_mape['pcnt25spd'] = test_mape25
test_mape['pcnt30spd'] = test_mape30
test_mape['pcnt35spd'] = test_mape35
test_mape['pcnt40spd'] = test_mape40
test_mape['pcnt45spd'] = test_mape45
test_mape['pcnt50spd'] = test_mape50
test_mape['pcnt55spd'] = test_mape55
test_mape['pcnt60spd'] = test_mape60
test_mape['pcnt65spd'] = test_mape65
test_mape['pcnt70spd'] = test_mape70
test_mape['pcnt75spd'] = test_mape75
test_mape['pcnt80spd'] = test_mape80
test_mape['pcnt85spd'] = test_mape85
test_mape['pcnt90spd'] = test_mape90
test_mape['pcnt95spd'] = test_mape95
test_mape['pcnt99spd'] = test_mape99

In [ ]:
test_rmse.to_csv('freeway_xgblss_test_rmse.csv', index=False)
test_mae.to_csv('freeway_xgblss_test_mae.csv', index=False)
test_mape.to_csv('freeway_xgblss_test_mape.csv', index=False)

In [ ]:
test_rmse

In [ ]:
test_mae

In [ ]:
test_mape